# Chat Model의 도구 호출 

- 도구는 에이전트, 체인 또는 대형 언어 모델(LLM)이 세상과 상호 작용할 수 있게 하는 인터페이스입니다.
    - Python REPL, Wikipdedia, YouTube, Zapier, Gradio, etc

In [1]:
# env 파일에서 API 키를 로드
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-nano", model_provider="openai")
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

### 도구 스키마 정의

모델이 도구를 호출할 수 있으려면, 그 도구가 무엇을 하는지와 어떤 인자를 받는지 설명하는 **도구 스키마(tool schema)**를 모델에 전달해야 합니다.
도구 호출 기능을 지원하는 채팅 모델은 .bind_tools() 메서드를 구현하고 있으며, 이 메서드를 사용해 도구 스키마를 모델에 넘길 수 있습니다.

도구 스키마는 다음과 같은 방식으로 정의할 수 있습니다:

- 타입 힌트와 독스트링이 포함된 파이썬 함수  
- Pydantic 모델   
- TypedDict 클래스  
- LangChain Tool 객체

이후 모델을 호출할 때마다, 해당 도구 스키마들이 프롬프트와 함께 전달됩니다.

### Python 함수
함수 이름, 타입 힌트, 그리고 독스트링(docstring)은 모델에 전달되는 도구 스키마(tool schema)의 일부입니다.  
좋은 설명을 가진 스키마를 정의하는 것은 프롬프트 엔지니어링의 확장이라고 할 수 있으며, 모델의 성능을 잘 끌어내는 데 중요한 부분입니다. 

**@tool** 데코레이터는 커스텀 도구를 정의하는 가장 간단한 방법입니다.  
이 데코레이터는 기본적으로 함수 이름을 도구 이름으로 사용하지만, 첫 번째 인자로 문자열을 전달하면 이를 덮어쓸 수 있습니다. 
또한 함수의 독스트링(docstring)을 도구의 설명(description) 으로 사용하므로, docstring은 반드시 제공되어야 합니다.

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool

# 1단계 : 도구 정의
@tool
def add(a: int, b: int) -> int:
    """두 정수를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """두 정수를 곱합니다."""
    return a * b

tools = [add, multiply]
tools

[StructuredTool(name='add', description='두 정수를 더합니다.', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x114aa3240>),
 StructuredTool(name='multiply', description='두 정수를 곱합니다.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x114b02480>)]

In [4]:
# 2단계 - 도구 바인딩된 모델 생성
llm_with_tools = model.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11427ab50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x114b25b50>, root_client=<openai.OpenAI object at 0x114279b10>, root_async_client=<openai.AsyncOpenAI object at 0x114b25690>, model_name='gpt-5-nano', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': '두 정수를 더합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': '두 정수를 곱합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [5]:
# 3단계 - 모델 호출
query = "3 * 12 는 얼마이고 4 + 49 는 얼마인가요? 직접 계산하지 말고 주어진 도구를 사용하세요."

ai_msg: AIMessage = llm_with_tools.invoke(query)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qYKnXzehhQJ55dBr5MQfFDuW', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_hBsrbcEZqKDXgDjiQTIspviP', 'function': {'arguments': '{"a": 4, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 443, 'prompt_tokens': 181, 'total_tokens': 624, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMnLdoWeDXXODFw7D2ENKZRt6NcME', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--350ea325-5629-417a-89ea-1e2f6ec117cb-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qYKnXzehhQJ55dBr5MQfFDuW', 'type': 'tool_call'

In [6]:
# 4단계 - 도구 실행: 바로 위 ai_msg.tool_calls 사용 
tool_results: list[ToolMessage] = []

for tc in ai_msg.tool_calls:  
    if tc["name"] == "multiply":
        out = multiply.invoke(tc["args"])
    elif tc["name"] == "add":
        out = add.invoke(tc["args"])
    else:
        out = "Unsupported tool"
    # ai_msg.tool_calls의 'id' 값을 그대로 연결
    tool_results.append(ToolMessage(tool_call_id=tc["id"], content=str(out)))

# 5단계 - 최종 호출: 직전 AIMessage와 그에 매칭되는 ToolMessage들을 연속으로 전달
summary_prompt = [
    HumanMessage(content=query),  # 사용자의 질문
    ai_msg,                       # 도구 호출 지시
    *tool_results                 # 도구 실행 결과
]

final_response = model.invoke(summary_prompt)
print(final_response.content)

결과는 다음과 같습니다.
- 3 × 12 = 36
- 4 + 49 = 53

도구를 사용해 계산했습니다.


### LangChin 내장 도구 사용

In [7]:
# !pip install --upgrade -q youtube_search

In [8]:
# 유튜브를 검색하는 함수
from langchain_community.tools import YouTubeSearchTool

# YouTubeSearchTool 인스턴스 생성
youtube_tool = YouTubeSearchTool()

# 아무 키워드로 유튜브 검색 실행 --> 함수가 잘 실행되는 것 확인.
results = youtube_tool.run("코딩하는 AI")
print(results)

['https://www.youtube.com/watch?v=O5-cNXWWdzs&pp=ygUP7L2U65Sp7ZWY64qUIEFJ0gcJCfsJAYcqIYzv', 'https://www.youtube.com/shorts/KzewB6tX8p8']


- YouTubeSearchTool 함수를 LLM 에 넘겨 주고 LLM이 이 함수를 이용하여 세상과 상호 작용 (유튜브 검색)할 수 있도록 함

In [9]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# 1단계: YouTube 도구를 LLM에 바인딩 
# OpenAI API 의 'tools=' parameter에 함수명과 
# 함수 호출에 필요한 parameter 정보 전달
llm_with_tools = model.bind_tools([youtube_tool])

system_message = SystemMessage(
    content="너는 질문에 답하기 위해 주어진 도구를 활용해야 해. 주어진 도구를 사용해서 정확한 정보를 찾아 응답해."
)

user_message = HumanMessage(content="'코딩하는 AI' 검색어의 유튜브 링크를 추천해줘")

ai_msg = llm_with_tools.invoke([system_message, user_message])
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_862SbpnkG7YLXfyeSd7qOOrr', 'function': {'arguments': '{"query":"코딩하는 AI, 5"}', 'name': 'youtube_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 733, 'prompt_tokens': 222, 'total_tokens': 955, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMnLkW23H0SYJ8SwBIwnrnT4LSF2g', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c6fba199-81dc-4cae-86e2-04a62a146191-0', tool_calls=[{'name': 'youtube_search', 'args': {'query': '코딩하는 AI, 5'}, 'id': 'call_862SbpnkG7YLXfyeSd7qOOrr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 222, 'output_tokens': 733, 'total_tokens': 955, 'input_token_deta

In [10]:
# msg에서 LLM이 구성해준 함수호출 parameter 발췌
ai_msg.tool_calls

[{'name': 'youtube_search',
  'args': {'query': '코딩하는 AI, 5'},
  'id': 'call_862SbpnkG7YLXfyeSd7qOOrr',
  'type': 'tool_call'}]

In [11]:
# YouTubeSearchTool 호출에 필요한 parameter
ai_msg.tool_calls[0]["args"]

{'query': '코딩하는 AI, 5'}

In [12]:
# 2단계 - chain 작성 
# llm_with_tools에서 추출된 인수로 YouTubeSearchTool을 사용하여 유튜브 검색 실행하는 chain 
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | youtube_tool
chain

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11427ab50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x114b25b50>, root_client=<openai.OpenAI object at 0x114279b10>, root_async_client=<openai.AsyncOpenAI object at 0x114b25690>, model_name='gpt-5-nano', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'youtube_search', 'description': 'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])
| RunnableLambda(lambda x: x.tool_calls[0]['args'])
| YouTubeSearchTool()

In [13]:
# chain 실행
search_results = chain.invoke([system_message, user_message])
search_results

"['https://www.youtube.com/watch?v=O5-cNXWWdzs&pp=ygUP7L2U65Sp7ZWY64qUIEFJ', 'https://www.youtube.com/watch?v=y9JSyGbulzk&pp=ygUP7L2U65Sp7ZWY64qUIEFJ']"

In [14]:
# 3.단계: 모든 중간 history와 최종 검색 결과를 다시 LLM에 전달하여 요약 요청
from langchain_core.messages import ToolMessage

summary_prompt = [
    system_message,
    user_message,
    ai_msg,
]

# 도구 실행 결과를 ToolMessage로 추가
for tc in ai_msg.tool_calls:
    if tc["name"] == "youtube_search":
        # 실제 실행 결과 search_results를 그대로 넣음
        summary_prompt.append(
            ToolMessage(
                tool_call_id=tc["id"],   # 반드시 ai_msg.tool_calls의 id 사용
                content=str(search_results)
            )
        )

# 추가 사용자 지시: 요약 요청
summary_prompt.append(HumanMessage(content="위 검색 결과를 요약해서 추천해줘."))

# 최종 응답 생성
final_response = model.invoke(summary_prompt)
print(final_response.content)

링크 두 개를 확인했어요. 다만 영상의 제목이나 상세 설명을 바로 확인할 수 없어서, 정확한 요약은 제가 조금 더 정보가 필요합니다. 현재로서는 두 영상이 “코딩에 AI를 활용하는 방법”에 대해 다루는 일반적인 콘텐츠일 가능성이 큽니다.

대략적인 요약 가능 내용(일반적인 코딩하는 AI 영상의 구성)
- AI를 활용한 코드 작성, 보조 도구 소개(Copilot, Codeium 등)와 워크플로우 예제
- AI의 장점과 한계, 실무 적용 시 주의점(신뢰도, 보안, 코드 품질 이슈)
- 간단한 실습 예제나 데모: AI로 코드 생성/수정/디버깅하는 과정
- 도구 비교나 실무 팁(언어/프레임워크별 활용 방법)

추천 방식
- 초보자라면: 환경 구성과 기본 사용법, 간단한 예제로 시작하는 영상이 더 유익할 가능성이 큽니다.
- 좀 더 실무적으로 배우고 싶으면: 실제 프로젝트에 AI를 어떻게 적용하는지, 특정 도구나 워크플로우를 심도 있게 다루는 영상이 좋습니다.

원하시면 더 정확하게 요약해 드리겠습니다. 두 영상의 제목/설명이나 길이 정보를 여기에 붙여 주시거나, 제가 영상의 제목을 확인해 요약해 달라고 허용해 주시면 바로 구체적으로 정리해서 추천해 드릴게요.
